In [48]:
import io
from shareplum import Site
from shareplum import Office365
from shareplum.site import Version
from unidecode import unidecode
import pandas as pd
from thefuzz import fuzz
import requests
import numpy as np

#  Carga de Archivos Manuales

In [49]:
# Ruta del archivo de Excel
file_path = r"C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\00.1 SO Enero\SOJunio.xlsx"
file_path2 = r"C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\00.1 SO Enero\EAN_Mapeo.xlsx"
file_path3 = r"C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\00.0 Dimensionales\GPS.xlsx"
file_path4 = r'C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\00.0 Dimensionales\Buscador EAN.xlsx'
# Cargar el archivo de Excel en un DataFrame
df = pd.read_excel(file_path)
ean_mapeo= pd.read_excel(file_path2)
gps= pd.read_excel(file_path3)
ean= pd.read_excel(file_path4)
gps_cliente= pd.read_excel(file_path3,sheet_name='CLIENTES')

In [50]:
# ean_mapeo= pd.read_excel(file_path2)
ean_mapeo['Ean'].fillna(0, inplace=True)
ean_mapeo['Ean']=ean_mapeo['Ean'].astype(np.int64)
ean_mapeo['Ean Correcto'].fillna(0, inplace=True)
ean_mapeo['Ean Correcto']=ean_mapeo['Ean Correcto'].astype(np.int64)
# ean_mapeo.info()

In [51]:
df['Stock'].fillna(0, inplace=True)
df['Stock']=df['Stock'].astype(np.int64)
df['Importe Neto'].fillna(0, inplace=True)
df['Importe Neto']=df['Importe Neto'].astype(np.float64)
df_historico=df.copy()
df_historico.loc[df["Fecha"].isna(), "Fecha"] = pd.to_datetime("1/5/2025", format="%d/%m/%Y")
df_historico["Fecha"] = pd.to_datetime(df_historico["Fecha"], errors='coerce', dayfirst=True)
df_historico["Fecha"] = pd.to_datetime(df_historico["Fecha"], format="%d/%m/%Y")
# df.info()

In [52]:
gps['Cliente'].fillna(0, inplace=True)
gps['Cliente']=gps['Cliente'].astype(int)
# gps.info()

In [53]:
# gps_cliente= pd.read_excel(file_path3,sheet_name='CLIENTES')

# Carga de Cambios

In [54]:
ean_historico = ean_mapeo[(ean_mapeo['Ean Correcto'] > 0) | ((ean_mapeo['Ean Correcto'] == 0) & (ean_mapeo['Tester'] == 'Si'))]
ean_historico_errors = ean_mapeo[(ean_mapeo['Ean Correcto'] < 0) | ((ean_mapeo['Ean Correcto'] != 0) & (ean_mapeo['Tester'] == 'Si'))]
ean_historico_unico = ean_historico.drop_duplicates(subset=['Desc Prod'], keep='first')
# ean_historico
df_corregido = pd.merge(
    df,
    ean_historico_unico[['Desc Prod', 'Ean Correcto', 'Tester']],
    on='Desc Prod',
    how='left'
)
df_corregido['Ean Correcto'].fillna(0, inplace=True)
df_corregido['Ean Correcto'] = df_corregido['Ean Correcto'].astype(np.int64)
df_corregido = df_corregido[df_corregido['Tester'] != 'Si']
# df_corregido['Ean'] = df_corregido['Ean'].fillna(0)
# df_corregido["Ean sist"] = df_corregido["Ean"].isin(ean["EAN"])
# df_corregido["Ean LRC"] = df_corregido["LRC"].isin(ean["LRC"])
df_corregido.loc[df_corregido['Ean Correcto'] > 0, 'Ean'] = df_corregido.loc[df_corregido['Ean Correcto'] > 0, 'Ean Correcto']
# df_corregido.loc[
#     (df_corregido['Ean Correcto'] > 0) & (df_corregido['Ean sist'] == False),
#     'Ean'
# ] = df_corregido.loc[
#     (df_corregido['Ean Correcto'] > 0) & (df_corregido['Ean sist'] == False),
#     'Ean Correcto'
# ]
df_corregido = df_corregido.copy()
df_corregido.drop(columns=['Ean Correcto', 'Tester'], inplace=True)
# df_corregido

In [55]:
df=df_corregido.copy()

In [56]:
# df

In [57]:
# df['Ean'] = df['Ean'].astype(int)


# Transformaciones

In [58]:
df["Tipo de Venta"]="Sell out"
# agregar fechas de archivo y completar los nulos
# df["Fecha"]="1/1/2025"
df.loc[df["Fecha"].isna(), "Fecha"] = pd.to_datetime("1/5/2025", format="%d/%m/%Y")
df["Fecha"] = pd.to_datetime(df["Fecha"], errors='coerce', dayfirst=True)
df["Fecha"] = pd.to_datetime(df["Fecha"], format="%d/%m/%Y")

In [59]:
df['Ean'] = df['Ean'].fillna(0)
df["Ean sist"] = df["Ean"].isin(ean["EAN"])
df["Ean LRC"] = df["LRC"].isin(ean["LRC"])

In [60]:
# df.info()

In [61]:
gps_duplicates = gps.duplicated(subset=["Codigo cliente", "Cliente"], keep=False)
# gps[gps_duplicates]

In [62]:
# Normalizar una columna específica (minúsculas y sin acentos)
def normalizar_columna(col):
    col.astype(str)
    return col.str.lower().apply(unidecode).str.strip()

gps_unico = gps.drop_duplicates(subset=["Codigo cliente", "Cliente"], keep='first')

df_merged = pd.merge(
    df,
    gps_unico[["Código Coty","Codigo cliente","Cliente","Ecommerce"]],
    left_on=["Punto de Venta","Grupo economico"],
    right_on=["Codigo cliente","Cliente"],
    how='left',
    indicator=True
)

In [63]:
df_merged2 = pd.merge(
    df_merged,
    gps_cliente[["ID_CLIENTE","Nombre Correcto"]],
    left_on=["Grupo economico"],
    right_on=["ID_CLIENTE"],
    how='left'
    # indicator=True
)

In [64]:
# Lista de columnas a eliminar
columnas_a_eliminar = ["Cliente","ID_CLIENTE"]

# Eliminar columnas del DataFrame
df_merged2 = df_merged2.drop(columns=columnas_a_eliminar)
df=df_merged2

In [65]:
df = df.rename(columns={"Nombre Correcto": "Grupo economico Final"})
df = df.rename(columns={"Código Coty": "PDV GPS"})

In [66]:
df["Punto de Venta"] = df["Punto de Venta"].astype(str)
gps["Codigo cliente"] = gps["Codigo cliente"].astype(str)


# Transformaciones Particulares para Clientes Específicos

In [67]:
df['Importe Neto'] = df['Importe Neto'].astype(float)

## Restar Iva al importe

In [68]:
df.loc[df['Grupo economico Final'] == 'FREE SHOP S.A.', 'Importe Neto'] = df['Importe Neto'] / 1.21
df.loc[df['Grupo economico Final'] == 'GRUPO JUFEC', 'Importe Neto'] = df['Importe Neto'] / 1.21
df.loc[df['Grupo economico Final'] == 'PERFUGROUP', 'Importe Neto'] = df['Importe Neto'] / 1.21
df.loc[df['Grupo economico'] == '53', 'Importe Neto'] = df['Importe Neto'] / 1.21
df.loc[df['Grupo economico'] == '82', 'Importe Neto'] = df['Importe Neto'] / 1.21

## Completar PDV con Grupo

In [69]:
df.loc[df['Grupo economico Final'] == 'DROGUERIA 20 DE JUNIO S.A.', ['PDV GPS', 'Ecommerce']] = ['DROGUERIA 20 DE JUNIO S.A.',False]
# df.loc[df['Grupo economico Final'] == 'DROGUERIA 20 DE JUNIO S.A.', 'PDV GPS'] = 'DROGUERIA 20 DE JUNIO S.A.'
df.loc[df['Grupo economico'] == 167, ['PDV GPS', 'Ecommerce']] = ['COFARSUR S.A.',False]
df.loc[df['Grupo economico Final'] == 'FIRST LABEL SRL', ['PDV GPS', 'Ecommerce']] = ['FIRST LABEL SRL.', True]
df.loc[df['Grupo economico Final'] == 'NIPPON', ['PDV GPS', 'Ecommerce']] = ['Nippon Nippon', False]
df.loc[df['Grupo economico Final'] == 'DROGUERIA SUR S.A.', ['PDV GPS', 'Ecommerce']] = ['DROGUERIA SUR S.A.', False]
df.loc[df['Grupo economico Final'] == 'DROGUERIA SUIZO', ['PDV GPS', 'Ecommerce']] = ['DROGUERIA SUIZO', False]
df.loc[df['Grupo economico Final'] == 'FARMAX SRL', ['PDV GPS', 'Ecommerce']] = ['FARMAX SRL', False]
df.loc[df['Grupo economico Final'] == 'TIENDA LOS GALLEGOS', ['PDV GPS', 'Ecommerce']] = ['TIENDA LOS GALLEGOS', False]
df.loc[df['Grupo economico Final'] == 'PRODEFARM S.A.', ['PDV GPS', 'Ecommerce']] = ['PRODEFARM S.A.', False]
df.loc[df['Grupo economico'] == 32, ['PDV GPS', 'Ecommerce']] = ['PERFUMERIAS ALDAZ', False]
df.loc[df['Grupo economico'] == 173, ['PDV GPS', 'Ecommerce']] = ['CASTRO JOSE MANUEL Y OTRO SOC.DE HECHO', False]
df.loc[df['Grupo economico'] == 166, ['PDV GPS', 'Ecommerce']] = ['KELLER', False]
df.loc[df['Grupo economico'] == 99, ['PDV GPS', 'Ecommerce']] = ['TIENDA LOS GALLEGOS', False]
df.loc[df['Grupo economico'] == 32, ['PDV GPS', 'Ecommerce']] = ['PREFUMERIAS ALDAZ', False]
df.loc[df['Grupo economico'] == 174, ['PDV GPS', 'Ecommerce']] = ['MAGLIONE', False]
df.loc[df['Grupo economico'] == 175, ['PDV GPS', 'Ecommerce']] = ['COOP.FARMACEUTICA MENDOZA LTDA.', False]
df.loc[df['Grupo economico'] == 176, ['PDV GPS', 'Ecommerce']] = ['ASOPROFARMA C.P.L.', False]
df.loc[df['Grupo economico'] == 177, ['PDV GPS', 'Ecommerce']] = ['ADM FRAGANCIAS S.R.L.', False]
df.loc[df['Grupo economico'] == 178, ['PDV GPS', 'Ecommerce']] = ['CASA DOAN', False]
df.loc[df['Grupo economico'] == 179, ['PDV GPS', 'Ecommerce']] = ['FARMAQUEN', False]
df.loc[df['Grupo economico'] == 180, ['PDV GPS', 'Ecommerce']] = ['FARMASUR S.R.L.', False]
df.loc[df['Grupo economico'] == 181, ['PDV GPS', 'Ecommerce']] = ['HERCOM S.A.S', False]







In [70]:
df.loc[df['Grupo economico'] == 20, 'Punto de Venta'] = df.loc[df['Grupo economico'] == 20, 'Punto de Venta'].str.replace('.1', '')


C:\Users\ArnaldoCarrasco\AppData\Local\Temp\ipykernel_8860\3289516671.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[df['Grupo economico'] == 20, 'Punto de Venta'] = df.loc[df['Grupo economico'] == 20, 'Punto de Venta'].str.replace('.1', '')


# Manejo de Errores

In [71]:
# df_filtrado = df[df["Grupo economico Final"] == "PRAFFEM S.R.L"]
# df_filtrado
# suma_cantidad = df_filtrado["Cantidad"].sum()
# suma_cantidad

In [72]:
ean_errors= df[df['Ean sist']==False]
# ean_errors

In [73]:
ean_errors_unique = ean_errors[["Ean", "Marca", "Desc Prod","Grupo economico Final"]].drop_duplicates().values.tolist()
ean_errors_unique=pd.DataFrame(ean_errors_unique,columns=["Ean", "Marca", "Desc Prod","Grupo economico Final"])
ean_errors_unique


,Ean,Marca,Desc Prod,Grupo economico Final


In [74]:
gps_errors=df[df['_merge']=='left_only']
gps_errors_unique = gps_errors[["Grupo economico", "Grupo economico Final","Punto de Venta","Codigo cliente", "PDV GPS"]].drop_duplicates().values.tolist()
gps_errors_unique= pd.DataFrame(gps_errors_unique,columns=["Grupo economico", "Grupo economico Final","Punto de Venta","Codigo cliente", "PDV GPS"])
gps_errors_unique

,Grupo economico,Grupo economico Final,Punto de Venta,Codigo cliente,PDV GPS
0,164,FIRST LABEL SRL,nan,NaN,FIRST LABEL SRL.
1,177,ADM FRAGANCIAS S.R.L.,nan,NaN,ADM FRAGANCIAS S.R.L.
2,168,DROGUERIA SUR S.A.,nan,NaN,DROGUERIA SUR S.A.
3,174,MARIA UÑA DE CARLETTO,nan,NaN,MAGLIONE
4,178,CASA DOAN,nan,NaN,CASA DOAN
5,169,DROGUERIA SUIZO,nan,NaN,DROGUERIA SUIZO
6,99,TIENDA LOS GALLEGOS,nan,NaN,TIENDA LOS GALLEGOS
7,179,GRUPO GLOBAL,nan,NaN,FARMAQUEN
8,181,HERCOM S.A.S,nan,NaN,HERCOM S.A.S
9,170,FARMAX SRL,nan,NaN,FARMAX SRL


In [75]:
ean_output_mapeo = pd.concat([ean_errors_unique, ean_historico], ignore_index=True)
ean_output_mapeo['Ean Correcto'] = ean_output_mapeo['Ean Correcto'].fillna(0)
ean_output_mapeo

,Ean,Marca,Desc Prod,Grupo economico Final,Ean Correcto,Tester
0,0,SALLY HANSEN (COTY),{SALLY HANSEN MIRACLE NAI 680,GRUPO RIÃDIGOS,7.417045e+10,NaN
1,88300190294,COTY LUXURY,CK ETERNITY WOMAN EDP VAPX100,GRUPO RIÃDIGOS,8.830019e+10,NaN
2,3616305447296,PROBADOR,PROBADOR ADIDAS HAPPY FEELS X 100,COFARSUR S.A.,0.000000e+00,Si
3,3616304892820,PROBADOR,PROBADOR DAVID BECKHAM INSTINCT EDP X 75,COFARSUR S.A.,0.000000e+00,Si
4,7795646128428,RIMMEL,RIMMEL EXH HIDRATATION BOOST MAYO,COFARSUR S.A.,0.000000e+00,Si
...,...,...,...,...,...,...
885,3616305447272,NaN,_PROBADOR ADIDAS VIBES FULL RECHARGE X 100,COFARSUR S.A.,0.000000e+00,Si
886,3616305447265,NaN,_PROBADOR ADIDAS VIBES ENERGY DRIVE X 100,COFARSUR S.A.,0.000000e+00,Si
887,3616305447302,NaN,_PROBADOR ADIDAS GET COMFY X 100,COFARSUR S.A.,0.000000e+00,Si
888,0,HUGO BOSS,HUGO MAN EDT 125 ML,DUTY PAID S.A.,7.370527e+11,NaN


In [76]:
df_conteo = ean_output_mapeo.groupby("Grupo economico Final").size().reset_index(name="Ean por mapear")
# df_conteo

ean_output_mapeo=ean_errors_unique
ean_output_mapeo["Ean Correcto"]=0

ean_output_mapeo = pd.concat([ean_output_mapeo, ean_historico])
ean_output_mapeo

,Ean,Marca,Desc Prod,Grupo economico Final,Ean Correcto,Tester
0,0,SALLY HANSEN (COTY),{SALLY HANSEN MIRACLE NAI 680,GRUPO RIÃDIGOS,74170450910,NaN
1,88300190294,COTY LUXURY,CK ETERNITY WOMAN EDP VAPX100,GRUPO RIÃDIGOS,88300191406,NaN
2,3616305447296,PROBADOR,PROBADOR ADIDAS HAPPY FEELS X 100,COFARSUR S.A.,0,Si
3,3616304892820,PROBADOR,PROBADOR DAVID BECKHAM INSTINCT EDP X 75,COFARSUR S.A.,0,Si
4,7795646128428,RIMMEL,RIMMEL EXH HIDRATATION BOOST MAYO,COFARSUR S.A.,0,Si
...,...,...,...,...,...,...
885,3616305447272,NaN,_PROBADOR ADIDAS VIBES FULL RECHARGE X 100,COFARSUR S.A.,0,Si
886,3616305447265,NaN,_PROBADOR ADIDAS VIBES ENERGY DRIVE X 100,COFARSUR S.A.,0,Si
887,3616305447302,NaN,_PROBADOR ADIDAS GET COMFY X 100,COFARSUR S.A.,0,Si
888,0,HUGO BOSS,HUGO MAN EDT 125 ML,DUTY PAID S.A.,737052713984,NaN


# Descargar archivo de salida EAN_MApeo

In [77]:
file_path = r'C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\Arvhivos de salida\EAN_Mapeo.xlsx'
ean_output_mapeo.to_excel(file_path, index=False)

# Filtrado de Grupos Economicos sin errores, y Validaciones de datos

In [78]:
tabla_estadisticas = df.groupby('Grupo economico Final').agg(
    Ean_erroneos=('Ean sist', lambda x: (x == False).any())
).reset_index()
tabla_estadisticas

,Grupo economico Final,Ean_erroneos
0,ACOFAR COOP.FARMACEUTICA LDA.,False
1,ADM FRAGANCIAS S.R.L.,False
2,ASOPROFARMA C.P.L.,False
3,CASA DOAN,False
4,COFARSUR S.A.,False
5,"CORTASSA ALDO, CORTASSA DIEGO,CORTASSA D",False
6,DROGUERIA DEL SUD S.A.,False
7,DROGUERIA SUIZO,False
8,DROGUERIA SUR S.A.,False
9,DUTY PAID S.A.,False


In [79]:
# tabla_status=tabla_estadisticas
# tabla_status["Ean_erroneos"] = tabla_status["Ean_erroneos"].replace({False: "ok", True: "Mapeo Pendiente"})
# tabla_status

In [80]:
grupos_con_falso = tabla_estadisticas.loc[tabla_estadisticas['Ean_erroneos'], 'Grupo economico Final'].tolist()
grupos_con_falso

[]

In [81]:
grupo_ok = df[~df['Grupo economico Final'].isin(grupos_con_falso)]
grupo_ok

,Fecha,Ean,Desc Prod,Grupo economico,Marca,Punto de Venta,Cantidad,Importe Neto,Stock,LRC,Tipo de Venta,Ean sist,Ean LRC,PDV GPS,Codigo cliente,Ecommerce,_merge,Grupo economico Final
0,2025-05-01,3.165553e+10,NAUTICA VOYAGE EDT 100 ML MASC,40,NAUTICA,AVLL,3,88040.08,0,NaN,Sell out,True,True,Avellaneda DDS,AVLL,False,both,DROGUERIA DEL SUD S.A.
1,2025-05-01,3.165553e+10,NAUTICA VOYAGE EDT 100 ML MASC,40,NAUTICA,BBCA,3,63337.39,0,NaN,Sell out,True,True,Bahía Blanca DDS,BBCA,False,both,DROGUERIA DEL SUD S.A.
2,2025-05-01,3.165553e+10,NAUTICA VOYAGE EDT 100 ML MASC,40,NAUTICA,CBJC,5,117457.17,0,NaN,Sell out,True,True,CBJC,CBJC,False,both,DROGUERIA DEL SUD S.A.
3,2025-05-01,3.165553e+10,NAUTICA VOYAGE EDT 100 ML MASC,40,NAUTICA,CDBA,0,0.00,0,NaN,Sell out,True,True,Córdoba DDS,CDBA,False,both,DROGUERIA DEL SUD S.A.
4,2025-05-01,3.165553e+10,NAUTICA VOYAGE EDT 100 ML MASC,40,NAUTICA,CTES,2,51643.97,0,NaN,Sell out,True,True,Corrientes DDS,CTES,False,both,DROGUERIA DEL SUD S.A.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188315,2025-04-01,7.417045e+10,SALLY H ESM TRAT INSTA-DRI TOP COAT,182,NaN,nan,10,0.00,0,NaN,Sell out,True,True,NaN,NaN,NaN,left_only,ACOFAR COOP.FARMACEUTICA LDA.
188316,2025-04-01,3.616303e+12,RIMMEL LAB LASTING PROVOCALIPS 220 COME UP ROESES,182,NaN,nan,12,0.00,0,NaN,Sell out,True,True,NaN,NaN,NaN,left_only,ACOFAR COOP.FARMACEUTICA LDA.
188317,2025-04-01,7.417045e+10,SALLY H ESM TRAT INSTA-DRI TOP COAT,182,NaN,nan,12,0.00,0,NaN,Sell out,True,True,NaN,NaN,NaN,left_only,ACOFAR COOP.FARMACEUTICA LDA.
188318,2025-04-01,7.891182e+12,RISQUE ESM 2 BASE FORTALECEDORA,182,NaN,nan,12,0.00,0,NaN,Sell out,True,True,NaN,NaN,NaN,left_only,ACOFAR COOP.FARMACEUTICA LDA.


# CSV Final

In [82]:
grupo_ok=df

In [83]:
grupo_ok = df[~df['Grupo economico Final'].isin(grupos_con_falso)]
grupo_ok = grupo_ok.drop(columns=["Desc Prod", "Grupo economico", "Marca", "Punto de Venta", "LRC", "Ean sist", "Ean LRC", "Codigo cliente", "_merge"])
grupo_ok['Importe Neto'] = grupo_ok['Importe Neto'].replace('nan', 0)

In [84]:
grupo_ok["Cantidad"] = grupo_ok["Cantidad"].replace([np.inf, -np.inf], np.nan).fillna(0).astype(int)
grupo_ok["Cantidad"] = grupo_ok["Cantidad"].astype(int)

In [85]:
# grupo_ok
grupo_ok.rename(columns={
    "PDV GPS": "Punto de Venta",
    "Grupo economico Final": "Grupo economico",
    "Ecommerce": "Columna1"
}, inplace=True)
grupo_ok["Columna1"] = grupo_ok["Columna1"].replace({True: "Si", False: "No"})
# grupo_ok

In [86]:
# file_path = r'C:\Users\ArnaldoCarrasco\Documents\COTY\SO SCRIPT\Arvhivos de salida\FebreroFinal.csv'
file_path2 = r'C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\Arvhivos de salida\MayoFinal.csv'
# so_final_febrero=  pd.read_csv(file_path)
# result = pd.concat([so_final_febrero, grupo_ok])
# 
# result

In [87]:
grupo_ok['Importe Neto']=grupo_ok['Importe Neto'].fillna(0)
grupo_ok['Importe Neto']=grupo_ok['Importe Neto'].astype(int)
grupo_ok

,Fecha,Ean,Cantidad,Importe Neto,Stock,Tipo de Venta,Punto de Venta,Columna1,Grupo economico
0,2025-05-01,3.165553e+10,3,88040,0,Sell out,Avellaneda DDS,No,DROGUERIA DEL SUD S.A.
1,2025-05-01,3.165553e+10,3,63337,0,Sell out,Bahía Blanca DDS,No,DROGUERIA DEL SUD S.A.
2,2025-05-01,3.165553e+10,5,117457,0,Sell out,CBJC,No,DROGUERIA DEL SUD S.A.
3,2025-05-01,3.165553e+10,0,0,0,Sell out,Córdoba DDS,No,DROGUERIA DEL SUD S.A.
4,2025-05-01,3.165553e+10,2,51643,0,Sell out,Corrientes DDS,No,DROGUERIA DEL SUD S.A.
...,...,...,...,...,...,...,...,...,...
188315,2025-04-01,7.417045e+10,10,0,0,Sell out,NaN,NaN,ACOFAR COOP.FARMACEUTICA LDA.
188316,2025-04-01,3.616303e+12,12,0,0,Sell out,NaN,NaN,ACOFAR COOP.FARMACEUTICA LDA.
188317,2025-04-01,7.417045e+10,12,0,0,Sell out,NaN,NaN,ACOFAR COOP.FARMACEUTICA LDA.
188318,2025-04-01,7.891182e+12,12,0,0,Sell out,NaN,NaN,ACOFAR COOP.FARMACEUTICA LDA.


In [88]:
grupo_ok.to_csv(file_path2, index=False)

In [89]:
df_filtrado = grupo_ok[grupo_ok['Grupo economico'] == 'CORTASSA ALDO, CORTASSA DIEGO,CORTASSA D']
df_filtrado

,Fecha,Ean,Cantidad,Importe Neto,Stock,Tipo de Venta,Punto de Venta,Columna1,Grupo economico
16255,2025-05-01,3.616305e+12,1,22561,0,Sell out,Rafaela Cortassa,No,"CORTASSA ALDO, CORTASSA DIEGO,CORTASSA D"
16256,2025-05-01,7.417044e+10,1,8842,0,Sell out,Rafaela Cortassa,No,"CORTASSA ALDO, CORTASSA DIEGO,CORTASSA D"
16257,2025-05-01,3.616302e+12,1,199338,0,Sell out,Santa Fe 1 Cortassa,No,"CORTASSA ALDO, CORTASSA DIEGO,CORTASSA D"
16258,2025-05-01,3.614230e+12,1,135000,0,Sell out,Santa Fe 1 Cortassa,No,"CORTASSA ALDO, CORTASSA DIEGO,CORTASSA D"
16259,2025-05-01,3.616303e+12,1,148500,0,Sell out,Santa Fe 1 Cortassa,No,"CORTASSA ALDO, CORTASSA DIEGO,CORTASSA D"
...,...,...,...,...,...,...,...,...,...
23479,2025-05-01,3.616304e+12,1,7896,0,Sell out,E-COMMERCE Cortassa,Si,"CORTASSA ALDO, CORTASSA DIEGO,CORTASSA D"
23480,2025-05-01,7.417045e+10,1,6545,0,Sell out,E-COMMERCE Cortassa,Si,"CORTASSA ALDO, CORTASSA DIEGO,CORTASSA D"
23481,2025-05-01,7.417045e+10,1,6545,0,Sell out,E-COMMERCE Cortassa,Si,"CORTASSA ALDO, CORTASSA DIEGO,CORTASSA D"
23482,2025-05-01,3.614222e+12,1,74380,0,Sell out,E-COMMERCE Cortassa,Si,"CORTASSA ALDO, CORTASSA DIEGO,CORTASSA D"


In [90]:
# utf_errors_path= r"C:\Users\ArnaldoCarrasco\Downloads\UTF.xlsx"
# utf_errors_path_out= r"C:\Users\ArnaldoCarrasco\Downloads\UTFfinal.csv"
# utf_errors= pd.read_excel(utf_errors_path,sheet_name='UTF')
# utf_errors.to_csv(utf_errors_path_out, index=False)
# 
# utf_errors

In [91]:
# grupos = grupo_ok.groupby(['Fecha', 'Grupo economico'])
# print(grupos)

In [92]:
# # Obtener una tabla final con columnas "Fecha" y "Grupo Economico" únicas
# status=grupo_ok
# status['Mes']= status['Fecha'].dt.month
# status['Año']= status['Fecha'].dt.year
# tabla_final = status.drop_duplicates(subset=['Grupo economico', 'Mes','Año'], keep='first')[['Año','Mes', 'Grupo economico']]
# 
# 
# tabla_final = tabla_final.sort_values(['Año', 'Mes', 'Grupo economico'])
# 
# # Mostrar la tabla final
# tabla_final

In [93]:
# Copias de seguridad
status = grupo_ok.copy()
status2 = df_historico.copy()

# ---- TRABAJO SOBRE STATUS (cantidad salida) ----
# Extraer mes y año
status['Mes'] = status['Fecha'].dt.month
status['Año'] = status['Fecha'].dt.year

# Agrupar por año, mes y grupo económico
tabla_salida = (
    status
    .groupby(['Año', 'Mes', 'Grupo economico'], as_index=False)
    .agg(Total_Cantidad_salida=('Cantidad', 'sum'))
)

# Asegurar tipos correctos
tabla_salida['Año'] = tabla_salida['Año'].astype(int)
tabla_salida['Mes'] = tabla_salida['Mes'].astype(int)
tabla_salida['Grupo economico'] = tabla_salida['Grupo economico'].astype(str)
tabla_salida['Total_Cantidad_salida'] = tabla_salida['Total_Cantidad_salida'].astype(np.int64)

# ---- TRABAJO SOBRE STATUS2 (cantidad original) ----
# Completar fechas nulas
status2.loc[status2['Fecha'].isna(), 'Fecha'] = pd.to_datetime('1/3/2025', format='%d/%m/%Y')

# Convertir fecha correctamente
status2['Fecha'] = pd.to_datetime(status2['Fecha'], errors='coerce', dayfirst=True)

# Extraer mes y año
status2['Mes'] = status2['Fecha'].dt.month
status2['Año'] = status2['Fecha'].dt.year

# Agrupar
tabla_original = (
    status2
    .groupby(['Año', 'Mes', 'Grupo economico'], as_index=False)
    .agg(Total_Cantidad_original=('Cantidad', 'sum'))
)

# Merge con gps_cliente para corregir nombre de grupo económico
tabla_original = pd.merge(
    tabla_original,
    gps_cliente[['ID_CLIENTE', 'Nombre Correcto']],
    left_on='Grupo economico',
    right_on='ID_CLIENTE',
    how='left'
)

# Limpiar columnas
tabla_original.drop(columns=['ID_CLIENTE', 'Grupo economico'], inplace=True)
tabla_original.rename(columns={'Nombre Correcto': 'Grupo economico'}, inplace=True)

# Asegurar tipos correctos
tabla_original['Año'] = tabla_original['Año'].astype(int)
tabla_original['Mes'] = tabla_original['Mes'].astype(int)
tabla_original['Grupo economico'] = tabla_original['Grupo economico'].astype(str)

# Reordenar columnas
tabla_original = tabla_original[['Año', 'Mes', 'Grupo economico', 'Total_Cantidad_original']]

# ---- MERGE FINAL ----
df_comparado = pd.merge(
    tabla_original,
    tabla_salida,
    on=['Año', 'Mes', 'Grupo economico'],
    how='outer'
)
df_comparado['Diferencia'] = (
        df_comparado['Total_Cantidad_salida'].fillna(0) -
        df_comparado['Total_Cantidad_original'].fillna(0)
)
df_comparado = df_comparado.sort_values(
    by=['Año', 'Mes', 'Grupo economico'],
    ascending=[True, True, True]
)

# Mostrar resultado
df_comparado    


,Año,Mes,Grupo economico,Total_Cantidad_original,Total_Cantidad_salida,Diferencia
0,2024,10,HERCOM S.A.S,358,358,0
1,2024,11,HERCOM S.A.S,282,282,0
2,2024,12,HERCOM S.A.S,685,685,0
4,2025,1,ACOFAR COOP.FARMACEUTICA LDA.,256,256,0
3,2025,1,HERCOM S.A.S,467,449,-18
6,2025,2,ACOFAR COOP.FARMACEUTICA LDA.,1621,1621,0
5,2025,2,HERCOM S.A.S,355,343,-12
8,2025,3,ACOFAR COOP.FARMACEUTICA LDA.,912,912,0
7,2025,3,HERCOM S.A.S,1142,1130,-12
10,2025,4,ACOFAR COOP.FARMACEUTICA LDA.,1024,1024,0


In [94]:
grupo_ok

,Fecha,Ean,Cantidad,Importe Neto,Stock,Tipo de Venta,Punto de Venta,Columna1,Grupo economico
0,2025-05-01,3.165553e+10,3,88040,0,Sell out,Avellaneda DDS,No,DROGUERIA DEL SUD S.A.
1,2025-05-01,3.165553e+10,3,63337,0,Sell out,Bahía Blanca DDS,No,DROGUERIA DEL SUD S.A.
2,2025-05-01,3.165553e+10,5,117457,0,Sell out,CBJC,No,DROGUERIA DEL SUD S.A.
3,2025-05-01,3.165553e+10,0,0,0,Sell out,Córdoba DDS,No,DROGUERIA DEL SUD S.A.
4,2025-05-01,3.165553e+10,2,51643,0,Sell out,Corrientes DDS,No,DROGUERIA DEL SUD S.A.
...,...,...,...,...,...,...,...,...,...
188315,2025-04-01,7.417045e+10,10,0,0,Sell out,NaN,NaN,ACOFAR COOP.FARMACEUTICA LDA.
188316,2025-04-01,3.616303e+12,12,0,0,Sell out,NaN,NaN,ACOFAR COOP.FARMACEUTICA LDA.
188317,2025-04-01,7.417045e+10,12,0,0,Sell out,NaN,NaN,ACOFAR COOP.FARMACEUTICA LDA.
188318,2025-04-01,7.891182e+12,12,0,0,Sell out,NaN,NaN,ACOFAR COOP.FARMACEUTICA LDA.
